In [1]:

# 1. Load CSV into DataFrame 
import pandas as pd

df = pd.read_excel("C:\\Users\\Admin\\Downloads\\Online_Retail.xlsx") 
print(df.head())
print(df.info())


  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       -----------

In [2]:
#data cleaning 
df.dropna(inplace=True)  # Remove rows with missing values
df = df[df['Quantity'] > 0]  # Keep only rows with positive Quantity
df = df[df['UnitPrice'] > 0]  # Keep only rows with positive UnitPrice
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])  # Convert InvoiceDate to datetime
df['CustomerID'] = df['CustomerID'].astype(str)  # Convert CustomerID to string
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  object        
 7   Country      397884 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.3+ MB
None
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029

In [3]:
# transformation
#calculate TotalSales
df['TotalSales'] = df['Quantity'] * df['UnitPrice']
#group by CustomerID to create a customer sales summary
customer_sales = df.groupby('CustomerID')['TotalSales'].sum().reset_index()
customer_sales = customer_sales.merge(df[['CustomerID', 'Country']].drop_duplicates(), on='CustomerID', how='left')
#filter data for sales in the las year
latest_date = df['InvoiceDate'].max()
one_year_ago = latest_date - pd.DateOffset(years=1)
recent_sales = df[df['InvoiceDate'] >= one_year_ago]
#handle outliers 
recent_sales = recent_sales[(recent_sales['Quantity'] > 0) & (recent_sales['UnitPrice'] > 0)]
print(customer_sales.head())
print(recent_sales.head())
#save transformed data
customer_sales.to_csv("customer_sales_summary.csv", index=False)
recent_sales.to_csv("recent_sales_data.csv", index=False)




  CustomerID  TotalSales         Country
0    12346.0    77183.60  United Kingdom
1    12347.0     4310.00         Iceland
2    12348.0     1797.24         Finland
3    12349.0     1757.55           Italy
4    12350.0      334.40          Norway
      InvoiceNo StockCode                         Description  Quantity  \
20240    538032     22669                   RED BABY BUNTING          5   
20241    538032     22465          HANGING METAL STAR LANTERN        12   
20242    538032    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
20243    538032     22727           ALARM CLOCK BAKELIKE RED          4   
20244    538032     22726          ALARM CLOCK BAKELIKE GREEN         4   

              InvoiceDate  UnitPrice CustomerID         Country  TotalSales  
20240 2010-12-09 12:59:00       2.95    14479.0  United Kingdom       14.75  
20241 2010-12-09 12:59:00       1.65    14479.0  United Kingdom       19.80  
20242 2010-12-09 12:59:00       2.95    14479.0  United Kingdom      

In [4]:
import sqlite3

# 1. Connect to SQLite database

conn = sqlite3.connect("retail_dw.db")
cursor = conn.cursor()


# 2. Create Dimension Tables


cursor.execute("""
CREATE TABLE IF NOT EXISTS CustomerDim (
    CustomerKey INTEGER PRIMARY KEY,
    CustomerID TEXT,
    CustomerName TEXT,
    CustomerGender TEXT,
    CustomerLocation TEXT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS TimeDim (
    TimeKey INTEGER PRIMARY KEY,
    Date TEXT,
    Day INTEGER,
    Month INTEGER,
    Year INTEGER,
    Quarter INTEGER
);
""")


# 3. Create Fact Table

cursor.execute("""
CREATE TABLE IF NOT EXISTS SalesFact (
    SalesKey INTEGER PRIMARY KEY AUTOINCREMENT,
    TimeKey INTEGER,
    CustomerKey INTEGER,
    ProductID TEXT,
    Quantity INTEGER,
    UnitPrice REAL,
    TotalAmount REAL,
    FOREIGN KEY (TimeKey) REFERENCES TimeDim(TimeKey),
    FOREIGN KEY (CustomerKey) REFERENCES CustomerDim(CustomerKey)
);
""")


# 4. Insert Sample Data (replace with your transformed data)


# Insert into CustomerDim 
customers = [
    (1, "C001", "John Doe", "Male", "Nairobi"),
    (2, "C002", "Mary Wanjiku", "Female", "Mombasa")
]

cursor.executemany("""
INSERT INTO CustomerDim (CustomerKey, CustomerID, CustomerName, CustomerGender, CustomerLocation)
VALUES (?, ?, ?, ?, ?)
""", customers)


# Insert into TimeDim 
times = [
    (1, "2025-01-01", 1, 1, 2025, 1),
    (2, "2025-01-02", 2, 1, 2025, 1)
]

cursor.executemany("""
INSERT INTO TimeDim (TimeKey, Date, Day, Month, Year, Quarter)
VALUES (?, ?, ?, ?, ?, ?)
""", times)


# Insert into SalesFact 
sales = [
    (1, 1, "P001", 2, 500.00, 1000.00),
    (2, 2, "P003", 1, 1200.00, 1200.00)
]

cursor.executemany("""
INSERT INTO SalesFact (TimeKey, CustomerKey, ProductID, Quantity, UnitPrice, TotalAmount)
VALUES (?, ?, ?, ?, ?, ?)
""", sales)

# Commit changes and close connection
conn.commit()
conn.close()

print("Retail Data Warehouse created and data loaded successfully!")



Retail Data Warehouse created and data loaded successfully!


In [5]:
#!/usr/bin/env python3
# ETL Pipeline for Retail Data Warehouse
import pandas as pd
import sqlite3
# 1. Load CSV into DataFrame
df = pd.read_excel("C:\\Users\\Admin\\Downloads\\Online_Retail.xlsx")
print(df.head())
print(df.info())
#data cleaning
df.dropna(inplace=True)  # Remove rows with missing values  
df = df[df['Quantity'] > 0]  # Keep only rows with positive Quantity
df = df[df['UnitPrice'] > 0]  # Keep only rows with positive UnitPrice
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])  # Convert InvoiceDate
df['CustomerID'] = df['CustomerID'].astype(str)  # Convert CustomerID to string
print(df.info())
print(df.head())
# transformation
#calculate TotalSales
df['TotalSales'] = df['Quantity'] * df['UnitPrice']
#group by CustomerID to create a customer sales summary 
customer_sales = df.groupby('CustomerID')['TotalSales'].sum().reset_index()
customer_sales = customer_sales.merge(df[['CustomerID', 'Country']].drop_duplicates(), on='CustomerID', how='left')
#filter data for sales in the las year
latest_date = df['InvoiceDate'].max()
one_year_ago = latest_date - pd.DateOffset(years=1)
recent_sales = df[df['InvoiceDate'] >= one_year_ago]
#handle outliers
recent_sales = recent_sales[(recent_sales['Quantity'] > 0) & (recent_sales['UnitPrice'] > 0)]

print(customer_sales.head())
print(recent_sales.head())      
#save transformed data
customer_sales.to_csv("customer_sales_summary.csv", index=False)
recent_sales.to_csv("recent_sales_data.csv", index=False)



  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       -----------